In [37]:
# Import required libraries
import ipywidgets as widgets
from IPython.display import display, HTML
import colorsys
from jinja2 import Template
import json

# Constants
MAX_COLORS = 12  # Maximum number of colors that can be generated

In [38]:
# ESPHome code template (Jinja2)
# Modify this template to customize the generated ESPHome configuration
ESPHOME_TEMPLATE = '''      - lambda:
          name: "{{ name }}"
          update_interval: {{ update_interval }}s
          # {{ config_json }}
          lambda: |-
            static int state = 0;
            auto call = id({{ light_id }}).turn_on();
            call.set_transition_length({{ transition_ms }});
            switch (state) {
{%- for i, color in enumerate(colors) %}
              case {{ i }}:
{%- if i == num_colors - 1 %}
              default:
{%- endif %}
                call.set_rgb({{ "%.1f" | format(color[0]) }}, {{ "%.1f" | format(color[1]) }}, {{ "%.1f" | format(color[2]) }});
                break;
{%- endfor %}
            }
            call.set_publish(true);
            call.set_save(false);
            call.perform();
            state += 1;
            if (state >= {{ num_colors }})
              state = 0;'''

In [39]:
# Create widgets
color_picker = widgets.ColorPicker(
    concise=False,
    description='Initial Color',
    value='#0000FF',
    disabled=False
)

hue_slider = widgets.FloatSlider(
    value=240,  # Blue starts at 240 degrees
    min=0,
    max=360,
    step=1,
    description='Hue (°):',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.0f'
)

saturation_slider = widgets.FloatSlider(
    value=100,  # Pure blue is 100% saturated
    min=0,
    max=100,
    step=1,
    description='Saturation (%):',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.0f'
)

brightness_slider = widgets.FloatSlider(
    value=100,  # Pure blue is 100% bright
    min=0,
    max=100,
    step=1,
    description='Brightness (%):',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.0f'
)

num_colors_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=MAX_COLORS,
    step=1,
    description='# Colors:',
    continuous_update=True,
    orientation='horizontal',
    readout=True
)

# Timing controls
transition_time = widgets.IntText(
    value=3000,
    description='Transition (ms):',
    disabled=False,
    style={'description_width': 'initial'}
)

update_interval = widgets.IntText(
    value=4000,
    description='Update Interval (ms):',
    disabled=False,
    style={'description_width': 'initial'}
)

# Progress bar to visualize timing
timing_progress = widgets.FloatProgress(
    value=75.0,  # 3000/4000 * 100
    min=0,
    max=100,
    description='Timing:',
    bar_style='info',
    orientation='horizontal'
)

timing_label = widgets.HTML(
    value='<div style="font-size: 0.9em; color: #666;">Transition: 3000ms | Update: 4000ms (75.0% transition)</div>'
)

# Configuration fields
effect_name = widgets.Text(
    value='Rainbow',
    description='Effect Name:',
    disabled=False,
    style={'description_width': 'initial'}
)

light_id = widgets.Text(
    value='my_light',
    description='Light ID:',
    disabled=False,
    style={'description_width': 'initial'}
)

# Output widget to display colors
output = widgets.Output()

In [40]:
# Flag to prevent circular updates
_updating = False

def hex_to_hsv(hex_color):
    """Convert hex color to HSV values (0-1 range)"""
    # Remove '#' if present
    hex_color = hex_color.lstrip('#')
    # Convert hex to RGB (0-1 range)
    r, g, b = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (0, 2, 4))
    # Convert RGB to HSV
    h, s, v = colorsys.rgb_to_hsv(r, g, b)
    return h, s, v

def hsv_to_hex(h, s, v):
    """Convert HSV values (0-1 range) to hex color"""
    # Convert HSV to RGB
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    # Convert RGB to hex
    return '#{:02x}{:02x}{:02x}'.format(int(r * 255), int(g * 255), int(b * 255))

def get_current_color():
    """Get the current color from sliders as HSV (0-1 range)"""
    h = hue_slider.value / 360.0  # Convert degrees to 0-1
    s = saturation_slider.value / 100.0  # Convert percentage to 0-1
    v = brightness_slider.value / 100.0  # Convert percentage to 0-1
    return h, s, v

def generate_colors(num_colors):
    """Generate evenly spaced colors on the hue scale"""
    h, s, v = get_current_color()
    
    colors = []
    if num_colors == 1:
        colors.append(hsv_to_hex(h, s, v))
    else:
        # Generate evenly spaced hues
        for i in range(num_colors):
            # Space colors evenly across the full hue range
            hue_offset = i / num_colors
            new_h = (h + hue_offset) % 1.0
            colors.append(hsv_to_hex(new_h, s, v))
    
    return colors

def update_timing_display(change=None):
    """Update the timing progress bar and label"""
    trans = transition_time.value
    interval = update_interval.value
    
    # Validate: transition must be less than update interval
    if trans > interval:
        timing_label.value = '<div style="font-size: 0.9em; color: red;">⚠️ Transition time must be less than update interval!</div>'
        timing_progress.bar_style = 'danger'
        return False
    
    # Calculate percentage
    percentage = (trans / interval * 100) if interval > 0 else 0
    
    # Update progress bar
    timing_progress.value = percentage
    timing_progress.bar_style = 'info'
    
    # Update label
    timing_label.value = f'<div style="font-size: 0.9em; color: #666;">Transition: {trans}ms | Update: {interval}ms ({percentage:.1f}% transition)</div>'
    
    return True

def update_display(change=None):
    """Update the color display"""
    with output:
        output.clear_output(wait=True)
        
        # Generate colors
        num_colors = num_colors_slider.value
        colors = generate_colors(num_colors)
        
        # Create HTML for color display
        html_parts = ['<div style="display: flex; gap: 0.5em; flex-wrap: wrap;">']
        
        for color in colors:
            html_parts.append(
                f'<div style="width: 3em; height: 2em; background-color: {color}; '
                f'border: 1px solid #333; border-radius: 3px;" '
                f'title="{color}"></div>'
            )
        
        html_parts.append('</div>')
        
        # Display colors
        display(HTML(''.join(html_parts)))
        
        # Display color codes
        color_list = ', '.join(colors)
        display(HTML(f'<div style="margin-top: 1em; font-family: monospace; font-size: 0.9em;">{color_list}</div>'))

def on_color_picker_change(change):
    """Update sliders when color picker changes"""
    global _updating
    if _updating:
        return
    
    _updating = True
    try:
        # Convert picker color to HSV
        h, s, v = hex_to_hsv(color_picker.value)
        
        # Update sliders
        hue_slider.value = h * 360.0  # Convert to degrees
        saturation_slider.value = s * 100.0  # Convert to percentage
        brightness_slider.value = v * 100.0  # Convert to percentage
        
        # Update display
        update_display()
    finally:
        _updating = False

def on_slider_change(change):
    """Update color picker when sliders change"""
    global _updating
    if _updating:
        return
    
    _updating = True
    try:
        # Get color from sliders
        h, s, v = get_current_color()
        
        # Update color picker
        color_picker.value = hsv_to_hex(h, s, v)
        
        # Update display
        update_display()
    finally:
        _updating = False

# Attach event handlers
color_picker.observe(on_color_picker_change, names='value')
hue_slider.observe(on_slider_change, names='value')
saturation_slider.observe(on_slider_change, names='value')
brightness_slider.observe(on_slider_change, names='value')
num_colors_slider.observe(update_display, names='value')
transition_time.observe(update_timing_display, names='value')
update_interval.observe(update_timing_display, names='value')

In [ ]:
# ESPHome code generation
def hex_to_rgb(hex_color):
    """Convert hex color to RGB values (0-1 range)"""
    hex_color = hex_color.lstrip('#')
    r, g, b = tuple(int(hex_color[i:i+2], 16) / 255.0 for i in (0, 2, 4))
    return r, g, b

def generate_esphome_code():
    """Generate ESPHome configuration code"""
    # Get all the values
    colors = generate_colors(num_colors_slider.value)
    num_colors = len(colors)
    
    # Convert colors to RGB
    rgb_colors = [hex_to_rgb(color) for color in colors]
    
    # Convert times to seconds
    update_interval_s = update_interval.value / 1000.0
    transition_ms = transition_time.value
    
    # Use the template constant defined earlier
    template = Template(ESPHOME_TEMPLATE)
    
    config_json = export_config()
    
    code = template.render(
        name=effect_name.value,
        update_interval=update_interval_s,
        transition_ms=transition_ms,
        light_id=light_id.value,
        colors=rgb_colors,
        num_colors=num_colors,
        enumerate=enumerate,
        config_json=config_json
    )
    
    return code

# Export/Import configuration
def export_config():
    """Export configuration as JSON string"""
    config = {
        "c": color_picker.value,
        "n": num_colors_slider.value,
        "tr": transition_time.value,
        "up": update_interval.value,
        "name": effect_name.value,
        "id": light_id.value
    }
    return json.dumps(config, indent=None, separators=(',', ':'))

def load_config(json_str):
    """Load configuration from JSON string"""
    global _updating
    try:
        config = json.loads(json_str)
        
        _updating = True
        
        # Update all widgets
        color_picker.value = config.get("c", "#0000FF")
        num_colors_slider.value = config.get("n", 3)
        transition_time.value = config.get("tr", 3000)
        update_interval.value = config.get("up", 4000)
        effect_name.value = config.get("name", "Rainbow")
        light_id.value = config.get("id", "my_light")
        
        _updating = False
        
        # Trigger updates
        on_color_picker_change(None)
        update_timing_display()
        
        return True, "Configuration loaded successfully!"
    except json.JSONDecodeError as e:
        return False, f"Error parsing JSON: {str(e)}"
    except Exception as e:
        return False, f"Error loading configuration: {str(e)}"

# Code output widget
code_output = widgets.Textarea(
    value='',
    placeholder='Generated ESPHome code will appear here',
    description='',
    disabled=False,
    layout=widgets.Layout(width='100%', height='400px'),
    style={'font_family': 'monospace'}
)

generate_button = widgets.Button(
    description='Generate ESPHome Code',
    button_style='success',
    tooltip='Generate configuration code',
    icon='check'
)

def on_generate_click(b):
    """Generate and display ESPHome code"""
    # Validate timing first
    if not update_timing_display():
        code_output.value = "Error: Transition time must be less than update interval!"
        return
    
    code = generate_esphome_code()
    code_output.value = code

generate_button.on_click(on_generate_click)

# Export/Import widgets
config_text = widgets.Textarea(
    value='',
    placeholder='JSON configuration will appear here for export, or paste JSON to import',
    description='',
    disabled=False,
    layout=widgets.Layout(width='100%', height='2em'),
    style={'font_family': 'monospace'}
)

export_button = widgets.Button(
    description='Export Config',
    button_style='info',
    tooltip='Export current configuration as JSON',
    icon='download'
)

load_button = widgets.Button(
    description='Load Config',
    button_style='warning',
    tooltip='Load configuration from JSON',
    icon='upload'
)

config_status = widgets.HTML(value='<div style="color: black; font-size: 0.9em;">...</div>')

def on_export_click(b):
    """Export configuration to JSON"""
    config_json = export_config()
    config_text.value = config_json
    config_status.value = '<div style="color: green; font-size: 0.9em;">✓ Configuration exported</div>'

def on_load_click(b):
    """Load configuration from JSON"""
    success, message = load_config(config_text.value)
    if success:
        config_status.value = f'<div style="color: green; font-size: 0.9em;">✓ {message}</div>'
    else:
        config_status.value = f'<div style="color: red; font-size: 0.9em;">✗ {message}</div>'

export_button.on_click(on_export_click)
load_button.on_click(on_load_click)

In [42]:
# Display all widgets
display(widgets.HTML('<h4 style="margin: 0.5em 0;">Color Selection</h4>'))
display(color_picker)
display(hue_slider)
display(saturation_slider)
display(brightness_slider)
display(num_colors_slider)
display(widgets.HTML('<hr style="margin: 1em 0;">'))
display(widgets.HTML('<h4 style="margin: 0.5em 0;">Color Preview</h4>'))
display(output)
display(widgets.HTML('<hr style="margin: 1em 0;">'))
display(widgets.HTML('<h4 style="margin: 0.5em 0;">Animation Timing</h4>'))
display(transition_time)
display(update_interval)
display(timing_progress)
display(timing_label)
display(widgets.HTML('<hr style="margin: 1em 0;">'))
display(widgets.HTML('<h4 style="margin: 0.5em 0;">Configuration</h4>'))
display(effect_name)
display(light_id)

# Display export/import section
display(widgets.HTML('<hr style="margin: 1em 0;">'))
display(widgets.HTML('<h4 style="margin: 0.5em 0;">Save/Load Configuration</h4>'))
display(widgets.HBox([export_button, load_button]))
display(config_text)
display(config_status)

# Display code generation section
display(widgets.HTML('<hr style="margin: 1em 0;">'))
display(widgets.HTML('<h4 style="margin: 0.5em 0;">ESPHome Code Generation</h4>'))
display(generate_button)
display(code_output)

# Initial displays
update_timing_display()
update_display()

HTML(value='<h4 style="margin: 0.5em 0;">Color Selection</h4>')

ColorPicker(value='#0000FF', description='Initial Color')

FloatSlider(value=240.0, description='Hue (°):', max=360.0, readout_format='.0f', step=1.0)

FloatSlider(value=100.0, description='Saturation (%):', readout_format='.0f', step=1.0)

FloatSlider(value=100.0, description='Brightness (%):', readout_format='.0f', step=1.0)

...

HTML(value='<div style="color: black; font-size: 0.9em;">...</div>')

HTML(value='<hr style="margin: 1em 0;">')

HTML(value='<h4 style="margin: 0.5em 0;">ESPHome Code Generation</h4>')

Button(button_style='success', description='Generate ESPHome Code', icon='check', style=ButtonStyle(), tooltip…

Textarea(value='', layout=Layout(height='400px', width='100%'), placeholder='Generated ESPHome code will appea…